In [ ]:
from ltr.client import ElasticClient
client = ElasticClient()

In [ ]:
from ltr import download
corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments.txt'

download([corpus, judgments], dest='data/');

In [ ]:
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

In [ ]:
client.reset_ltr(index='tmdb')

config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
                { #1
                    "name": "title_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"title": "{{keywords}}"}
                    }
                },
                { #2
                    "name": "overview_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"overview": "{{keywords}}"}
                    }
                }
            ]
    }}


client.create_featureset(index='tmdb', name='movies', ftr_config=config)

In [ ]:
from ltr.log import log_query
from ltr.judgments import judgments_open
from itertools import groupby

training_set=[]
with judgments_open('data/title_judgments.txt') as judgment_list:
    # For each query's judgments log features, and add to our training set...
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        query_training_set, discarded = log_query(client, 
                                                  index='tmdb', 
                                                  judgments=query_judgments, 
                                                  feature_set='movies')
        training_set.extend(query_training_set)
        print("Got %s Training Samples" % len(training_set))

In [ ]:
from ltr.ranklib import train
trainLog  = train(client,
                  training_set=training_set,
                  index='tmdb',
                  featureSet='movies',
                  modelName='title')

In [ ]:
from ltr import search
search(client, "cartoon with basketball aliens", modelName='title')